In [2]:
import _init_path
import argparse
import datetime
import glob
import os
from pathlib import Path
from test import repeat_eval_ckpt

import torch
import torch.nn as nn
from tensorboardX import SummaryWriter

from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.datasets import build_dataloader
from pcdet.models import build_network, model_fn_decorator
from pcdet.utils import common_utils
from train_utils.optimization import build_optimizer, build_scheduler
from train_utils.train_utils import train_model


#### 加载配置文件,文件定义了数据路径、类型个数、训练参数等等详细内容

In [3]:
cfg_file = "./cfgs/kitti_models/pointpillar.yaml"
cfg = cfg_from_yaml_file(cfg_file, cfg)

#### 加载模型

In [4]:
dist_train = False 
BatchSize = 1
Workers = 4
# -----------------------create dataloader & network & optimizer---------------------------
train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=BatchSize,
    dist=dist_train, workers=Workers,
    logger=None,
    training=True,
    merge_all_iters_to_one_epoch=False,
    total_epochs=None
)

model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=train_set)
# if args.sync_bn:
#     model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
model.cuda()
# model

/home/charles/anaconda3/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


PointPillar(
  (vfe): PillarVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): Linear(in_features=10, out_features=64, bias=False)
        (norm): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (backbone_3d): None
  (map_to_bev_module): PointPillarScatter()
  (pfe): None
  (backbone_2d): BaseBEVBackbone(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (2): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (8): BatchNorm2d(64

In [ ]:
optimizer = build_optimizer(model, cfg.OPTIMIZATION)

In [ ]:
# load checkpoint if it is possible
ckpt_dir = "./tempckpt/"
start_epoch = it = 0
last_epoch = -1

lr_scheduler, lr_warmup_scheduler = build_scheduler(
    optimizer, total_iters_each_epoch=len(train_loader), total_epochs=50,
    last_epoch=last_epoch, optim_cfg=cfg.OPTIMIZATION
)

In [31]:
model.train() 

train_model(
    model,
    optimizer,
    train_loader,
    model_func=model_fn_decorator(),
    lr_scheduler=lr_scheduler,
    optim_cfg=cfg.OPTIMIZATION,
    start_epoch=0,
    total_epochs=50,
    start_iter=it,
    rank=0,
    tb_log=None,
    ckpt_save_dir=ckpt_dir,
    train_sampler=train_sampler,
    lr_warmup_scheduler=lr_warmup_scheduler,
    ckpt_save_interval=1,
    max_ckpt_save_num=50,
    merge_all_iters_to_one_epoch=False
)

epochs:   0%|          | 0/50 [00:00<?, ?it/s]














epochs:   0%|          | 0/50 [00:00<?, ?it/s, loss=1.45, lr=0.0003, d_time=0.15(0.15), f_time=0.10(0.10), b_time=0.32(0.32)]









epochs:   0%|          | 0/50 [00:00<?, ?it/s, loss=1.28, lr=0.0003, d_time=0.01(0.08), f_time=0.06(0.08), b_time=0.13(0.22)]









epochs:   0%|          | 0/50 [00:01<?, ?it/s, loss=1.33, lr=0.0003, d_time=0.00(0.05), f_time=0.04(0.06), b_time=0.10(0.18)]









epochs:   0%|          | 0/50 [00:01<?, ?it/s, loss=1.43, lr=0.0003, d_time=0.00(0.04), f_time=0.04(0.06), b_time=0.11(0.16)]




epochs:   0%|          | 0/50 [00:01<?, ?it/s, loss=1.45, lr=0.0003, d_time=0.00(0.03), f_time=0.03(0.05), b_time=0.10(0.15)]









epochs:   0%|          | 0/50 [00:01<?, ?it/s, loss=1.27, lr=0.0003, d_time=0.00(0.03), f_time=0.04(0.05), b_time=0.10(0.14)]









epochs:   0%|          | 0/50 [00:01<?, ?it/s, loss=1.36, lr=0.0003, d_time=0.00(0.02), f_time=0.05(0.05), b_time=0.11(0.14)]








